In [6]:
SUBTASKS = [3, 2]
LANGUAGES = ["eng", "jpn", "rus", "tat", "ukr", "zho"]
DOMAINS = ["restaurant", "laptop", "hotel", "finance"]
STRATEGY = "train_split"  # "pred_dev" oder "train_split"
N_RUNS = 5 # Wie oft wurde prompt ausgeführt bei self-consistency

# Valid combinations of (language, domain) that have data
VALID_LANGUAGES_DOMAINS = [
    ("eng", "restaurant"),
    ("eng", "laptop"),
    ("jpn", "hotel"),
    ("rus", "restaurant"),
    ("tat", "restaurant"),
    ("ukr", "restaurant"),
    ("zho", "restaurant"),
    ("zho", "laptop"),
]

In [7]:
from collections import defaultdict
from helper import *
from evaluate import *
import pandas as pd
import os

In [8]:
def get_key_of_best_strategy(lang, domain, df):
    strategies = {
        "no_sc_guided": df.loc[(df["Language"] == language_mapping[lang]) & (df["Domain"] == domain_mapping[domain]), "no_sc_guided"].values[0],
        "no_sc_no_guided": df.loc[(df["Language"] == language_mapping[lang]) & (df["Domain"] == domain_mapping[domain]), "no_sc_no_guided"].values[0],
        "sc_guided": df.loc[(df["Language"] == language_mapping[lang]) & (df["Domain"] == domain_mapping[domain]), "sc_guided"].values[0],
        "sc_no_guided": df.loc[(df["Language"] == language_mapping[lang]) & (df["Domain"] == domain_mapping[domain]), "sc_no_guided"].values[0],
    }
    # throw error if any value is nan or np.float64(nan)
    for key in strategies:
        if pd.isna(strategies[key]):
            strategies[key] = None
    if all(value is None for value in strategies.values()):
        raise FileNotFoundError(
            f"No performance data found for language: {lang}, domain: {domain}")
    
    
    # Get strategy with highest score
    best_strategy = max(
        strategies, key=lambda k: strategies[k] if strategies[k] is not None else -1)
    
    return best_strategy


def get_performance_tabular(table_metric, table_subtask):
    table = defaultdict(lambda: defaultdict(dict))

    for language, domain in VALID_LANGUAGES_DOMAINS:
            try:
                performance = get_performance(
                    language, domain, table_subtask, STRATEGY)
                table[language][domain]["no_sc_guided"] = performance["no_sc_guided"][table_metric]
                table[language][domain]["no_sc_no_guided"] = performance["no_sc_no_guided"][table_metric]
                table[language][domain]["sc_guided"] = performance["sc_guided"][table_metric]
                table[language][domain]["sc_no_guided"] = performance["sc_no_guided"][table_metric]
            except FileNotFoundError:
                table[language][domain]["no_sc_guided"] = None
                table[language][domain]["no_sc_no_guided"] = None
                table[language][domain]["sc_guided"] = None
                table[language][domain]["sc_no_guided"] = None

    df_rows = []
    for language, domain in VALID_LANGUAGES_DOMAINS:
            row = {
                "Language": language_mapping[language],
                "Domain": domain_mapping[domain],
                "no_sc_guided": table[language][domain]["no_sc_guided"],
                "no_sc_no_guided": table[language][domain]["no_sc_no_guided"],
                "sc_guided": table[language][domain]["sc_guided"],
                "sc_no_guided": table[language][domain]["sc_no_guided"],
            }
            df_rows.append(row)
    df = pd.DataFrame(df_rows)

    # Add AVG row
    avg_row = {
        "Language": "AVG",
        "Domain": "",
        "no_sc_guided": df["no_sc_guided"].mean(skipna=True),
        "no_sc_no_guided": df["no_sc_no_guided"].mean(skipna=True),
        "sc_guided": df["sc_guided"].mean(skipna=True),
        "sc_no_guided": df["sc_no_guided"].mean(skipna=True),
    }

    df = pd.concat([df, pd.DataFrame([avg_row])], ignore_index=True)
    return df


df_subtask3 = get_performance_tabular("cF1", 3)
df_subtask2 = get_performance_tabular("cF1", 2)

df_subtasks = {
    3: df_subtask3,
    2: df_subtask2
}

In [9]:
df_subtask3

,Language,Domain,no_sc_guided,no_sc_no_guided,sc_guided,sc_no_guided
0,English,Restaurant,NaN,NaN,NaN,NaN
1,English,Laptop,NaN,NaN,NaN,NaN
2,Japanese,Hotel,NaN,NaN,NaN,NaN
3,Russian,Restaurant,NaN,NaN,NaN,NaN
4,Tatar,Restaurant,NaN,NaN,NaN,NaN
5,Ukrainian,Restaurant,NaN,NaN,NaN,NaN
6,Chinese,Restaurant,NaN,NaN,NaN,NaN
7,Chinese,Laptop,0.086944,0.082619,0.085076,0.072588
8,AVG,,0.086944,0.082619,0.085076,0.072588


In [10]:
get_key_of_best_strategy("eng", "restaurant", df_subtasks[3])

FileNotFoundError: No performance data found for language: eng, domain: restaurant

## Export Predictions in Valid Format

In [ ]:
strategy_export = "pred_dev"

In [ ]:
for subtask in SUBTASKS:
    for language, domain in VALID_LANGUAGES_DOMAINS:
        try:
            best_strategy = get_key_of_best_strategy(
                language, domain, df_subtasks[subtask])
            predictions = get_performance(language, domain, subtask, strategy_export, llm="unsloth/gemma-3-27b-it-bnb-4bit")[1][best_strategy]
            output_dir = f"exported_predictions/subtask_{subtask}/pred_{language}_{domain}.jsonl"
            os.makedirs(os.path.dirname(output_dir), exist_ok=True)
            with open(output_dir, "w", encoding="utf-8") as f:
                for pred in predictions:
                    f.write(json.dumps(pred, ensure_ascii=False) + "\n")
        except Exception as e:
            print(f"Error processing Subtask {subtask} - Language: {language}, Domain: {domain}: {e}")
            continue
        print(
            f"Subtask {subtask} - Language: {language}, Domain: {domain} => Best Strategy: {best_strategy}")

Error processing Subtask 3 - Language: eng, Domain: restaurant: [Errno 2] No such file or directory: 'results/results_pred_dev/unsloth_gemma-3-27b-it-bnb-4bit/3_eng_restaurant_0_temp0_with_guidance.jsonl'
Error processing Subtask 3 - Language: eng, Domain: laptop: No performance data found for language: eng, domain: laptop
Error processing Subtask 3 - Language: jpn, Domain: hotel: No performance data found for language: jpn, domain: hotel
Error processing Subtask 3 - Language: rus, Domain: restaurant: [Errno 2] No such file or directory: 'results/results_pred_dev/unsloth_gemma-3-27b-it-bnb-4bit/3_rus_restaurant_0_temp0_with_guidance.jsonl'
Error processing Subtask 3 - Language: tat, Domain: restaurant: [Errno 2] No such file or directory: 'results/results_pred_dev/unsloth_gemma-3-27b-it-bnb-4bit/3_tat_restaurant_0_temp0_with_guidance.jsonl'
Error processing Subtask 3 - Language: ukr, Domain: restaurant: [Errno 2] No such file or directory: 'results/results_pred_dev/unsloth_gemma-3-27b-

In [ ]:
import psutil
import os

process = psutil.Process(os.getpid())
ram_bytes = process.memory_info().rss
ram_mb = ram_bytes / (1024 * 1024)

print(f"RAM usage: {ram_mb:.2f} MB")


RAM usage: 740.25 MB


In [ ]:
"text " * 13

'text text text text text text text text text text text text text '